<a href="https://colab.research.google.com/github/ThisGuyEddie/aml_covid/blob/main/filter_COVID19_non_pharmaceutical_interventions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To Do List
* Need to join df and df_states
* Import all csv's in 

In [134]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re 
import numpy as np
import json
from datetime import datetime,date

# CSSEGIS Daily COVID Reports
Building a single dataframe from all csv files listed in [CSSEGIS Covid Data](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports).



In [135]:
# Store the url as a string scalar: url => str
url = "https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports"

# Issue request: r => requests.models.Response
r = requests.get(url)

# Extract text: html_doc => str
html_doc = r.text

# Parse the HTML: soup => bs4.BeautifulSoup
soup = BeautifulSoup(html_doc)

# Find all 'a' tags (which define hyperlinks): a_tags => bs4.element.ResultSet
a_tags = soup.find_all('a')

# Store a list of urls ending in .csv: urls => list
urls = ['https://raw.githubusercontent.com'+re.sub('/blob', '', link.get('href')) 
        for link in a_tags  if '.csv' in link.get('href')]

# Store a list of Data Frame names to be assigned to the list: df_list_names => list
df_list_names = [url.split('.csv')[0].split('/')[url.count('/')] for url in urls]

# Initialise an empty list the same length as the urls list
df_list = [pd.DataFrame([None]) for i in range(len(urls))]

# Create a list of DataFrames
df_list = [pd.read_csv(url, sep = ',') for url in urls]

# Join List of DataFrames into a single DataFrame
df_cssegis = pd.concat(df_list)
drop_columns = ['Lat','Long_','Recovered', 'Latitude','Longitude']
df_cssegis = df_cssegis.drop(columns=drop_columns)
df_cssegis = df_cssegis.rename({'Province_State':'State','Country/Region':'Country', 'Last_Update':'Date'})

df_cssegis['Last Update'].fillna(df_cssegis['Last_Update'])
df_cssegis.to_csv('CSSEGIS.csv',index=None)
df_cssegis

,Province/State,Country/Region,Last Update,Confirmed,Deaths,FIPS,Admin2,Province_State,Country_Region,Last_Update,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio,Incident_Rate,Case_Fatality_Ratio
0,Anhui,Mainland China,1/22/2020 17:00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Beijing,Mainland China,1/22/2020 17:00,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chongqing,Mainland China,1/22/2020 17:00,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fujian,Mainland China,1/22/2020 17:00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gansu,Mainland China,1/22/2020 17:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3969,NaN,NaN,NaN,61514.0,548.0,NaN,NaN,NaN,West Bank and Gaza,2020-11-14 05:25:37,7599.0,West Bank and Gaza,NaN,NaN,1205.822070,0.890854
3970,NaN,NaN,NaN,10.0,1.0,NaN,NaN,NaN,Western Sahara,2020-11-14 05:25:37,1.0,Western Sahara,NaN,NaN,1.674116,10.000000
3971,NaN,NaN,NaN,2072.0,605.0,NaN,NaN,NaN,Yemen,2020-11-14 05:25:37,73.0,Yemen,NaN,NaN,6.946966,29.198842
3972,NaN,NaN,NaN,17093.0,350.0,NaN,NaN,NaN,Zambia,2020-11-14 05:25:37,609.0,Zambia,NaN,NaN,92.977812,2.047622


In [ ]:
cssegis_countries = df_cssegis.Country_Region.unique()
cssegis_countries = json.dumps(sorted(cssegis_countries[1:]))
print(cssegis_countries)

["Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda", "Argentina", "Armenia", "Australia", "Austria", "Azerbaijan", "Bahamas", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize", "Benin", "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei", "Bulgaria", "Burkina Faso", "Burma", "Burundi", "Cabo Verde", "Cambodia", "Cameroon", "Canada", "Central African Republic", "Chad", "Chile", "China", "Colombia", "Comoros", "Congo (Brazzaville)", "Congo (Kinshasa)", "Costa Rica", "Cote d'Ivoire", "Croatia", "Cruise Ship", "Cuba", "Cyprus", "Czechia", "Denmark", "Diamond Princess", "Djibouti", "Dominica", "Dominican Republic", "Ecuador", "Egypt", "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia", "Eswatini", "Ethiopia", "Fiji", "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece", "Grenada", "Guatemala", "Guinea", "Guinea-Bissau", "Guyana", "Haiti", "Holy See", "Honduras", "Hungary", "Iceland", "I

Creating a set of filters for parsing the [NY Times COVID-19 Data](https://github.com/amel-github/covid19-interventionmeasures) data.

In [ ]:
#@title Specify State and Intervention Measure
url = 'https://raw.githubusercontent.com/amel-github/covid19-interventionmeasures/master/COVID19_non-pharmaceutical-interventions_version2_utf8.csv'
df = pd.read_csv(url, encoding='cp1252')

states_url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv'
df_states = pd.read_csv(states_url)

filter_country = False #@param {type: "boolean"}
country =  "United States" #@param {type:"string"}  ["Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda", "Argentina", "Armenia", "Australia", "Austria", "Azerbaijan", "Bahamas", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize", "Benin", "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei", "Bulgaria", "Burkina Faso", "Burma", "Burundi", "Cabo Verde", "Cambodia", "Cameroon", "Canada", "Central African Republic", "Chad", "Chile", "China", "Colombia", "Comoros", "Congo", "Costa Rica", "Cote d'Ivoire", "Croatia", "Cuba", "Cyprus", "Czechia", "Denmark", "Djibouti", "Dominica", "Dominican Republic", "Ecuador", "Egypt", "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia", "Eswatini", "Ethiopia", "Fiji", "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece", "Grenada", "Guatemala", "Guinea", "Guinea-Bissau", "Guyana", "Haiti", "Holy See", "Honduras", "Hungary", "Iceland", "India", "Indonesia", "Iran", "Iraq", "Ireland", "Israel", "Italy", "Jamaica", "Japan", "Jordan", "Kazakhstan", "Kenya", "Korea, South", "Kosovo", "Kuwait", "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia", "Libya", "Liechtenstein", "Lithuania", "Luxembourg", "MS Zaandam", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands", "Mauritania", "Mauritius", "Mexico", "Moldova", "Monaco", "Mongolia", "Montenegro", "Morocco", "Mozambique", "Namibia", "Nepal", "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria", "North Macedonia", "Norway", "Oman", "Pakistan", "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines", "Poland", "Portugal", "Qatar", "Romania", "Russia", "Rwanda", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines", "San Marino", "Sao Tome and Principe", "Saudi Arabia", "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Slovakia", "Slovenia", "Solomon Islands", "Somalia", "South Africa", "South Sudan", "Spain", "Sri Lanka", "Sudan", "Suriname", "Sweden", "Switzerland", "Syria", "Taiwan*", "Tajikistan", "Tanzania", "Thailand", "Timor-Leste", "Togo", "Trinidad and Tobago", "Tunisia", "Turkey", "United States", "Uganda", "Ukraine", "United Arab Emirates", "United Kingdom", "Uruguay", "Uzbekistan", "Vanuatu", "Venezuela", "Vietnam", "West Bank and Gaza", "Western Sahara", "Yemen", "Zambia", "Zimbabwe"]

if (filter_country == True):
    df = df.loc[df['Country'] == country]

# Dropping all data that is not from the USA
filter_for_us = False #@param {type: "boolean"}
if filter_for_us == True:
    df = df.loc[df['Country'] == 'United States of America']

# Filter Dataframe by State. 
filter_by_state = False #@param {type: "boolean"}
state = "New York" #@param ["Alaska", "Alabama", "Arkansas", "Arizona", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]
if(filter_by_state == True):
    df = df.loc[df['State'] == state]
    df_states = df_states.loc[df_states['state'] == state]

# Filter by safety measure.
filter_by_measure = False #@param {type: "boolean"}
measure = "Risk communication" #@param {type:"string"} ["Case identification, contact tracing and related measures", "Environmental measures", "Healthcare and public health capacity", "Resource allocation", "Risk communication", "Social distancing", "Travel restriction","Returning to normal life"]
if(filter_by_measure == True):
    df = df.loc[df['Measure_L1'] == measure]

# Create a range of dates and filter for that range. 
filter_by_date_range = False #@param {type:"boolean"}
start_date = "2020-03-02" #@param {type:"date"}
end_date = "2020-10-31" #@param {type:"date"}
date_range = pd.date_range(start_date, end_date)

df_states = df_states.rename(columns={'date':'Date','state':'State'})
if (filter_by_date_range == True):
    df = df[~df.Date.isin(date_range)]
    df_states = df_states[~df_states.Date.isin(date_range)]
df.to_csv('filtered_covid_measures.csv', index=None)
df_states.to_csv('filtered_us-states', index=None)

,id,Country,iso3,State,Region,Date,Measure_L1,Measure_L2,Measure_L3,Measure_L4,Status,Comment,Source
0,1,Albania,ALB,Albania,Albania,2020-02-25,Risk communication,Educate and actively communicate with the public,Encourage self-initiated quarantine,Travelers returning from high risk areas,NaN,"""The head of the National Medical Emergency Ce...",https://balkaneu.com/albania-national-medical-...
1,2,Albania,ALB,Albania,Albania,2020-02-25,Resource allocation,Crisis management plans,Financial aid for health system,Funds to increase availability of PPE,NaN,"""Albanian Ministry of Health and Social Protec...",https://balkaneu.com/albania-national-medical-...
2,3,Albania,ALB,Albania,Albania,2020-02-25,Healthcare and public health capacity,Adapt procedures for patient management,Implement triage procedures,Health hotline,NaN,"He clarified that when people notice symptoms,...",https://balkaneu.com/albania-national-medical-...
3,4,Albania,ALB,Albania,Albania,2020-02-25,"Case identification, contact tracing and relat...",Airport health check,Specific health channel for travellers,National passengers arriving from China,NaN,""" Albanian citizens arriving from China, Singa...",https://balkaneu.com/albania-national-medical-...
4,5,Albania,ALB,Albania,Albania,2020-03-08,Travel restriction,Airport restriction,Cancellation of international flights,Partial cancellation of international flights:...,NaN,Government stopped all flights with quarantine...,https://tvklan.al/ndalohen-fluturimet-me-10-de...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7008,7009,United States of America,USA,Wyoming,Wyoming,2020-03-25,Social distancing,Small gathering cancellation,Closure of non-essential shops,"Nail salons, hair salons, barber shops, massag...",NaN,"closure of nail salons, hair salons, barber sh...",https://health.wyo.gov/wp-content/uploads/2020...
7009,7010,United States of America,USA,Wyoming,Wyoming,2020-03-30,Social distancing,Small gathering cancellation,Closure of restaurants/bars/cafes,Partial: restaurants with liquor license can s...,NaN,Restaurant/Bar & Grill with liquor license all...,https://drive.google.com/file/d/1z_UrniG8E50dE...
7010,7011,United States of America,USA,Wyoming,Wyoming,2020-03-30,Risk communication,Actively communicate with managers,Guidelines,For grocery and retail stores,NaN,Guidance for Grocery and Retail Stores from th...,https://health.wyo.gov/wp-content/uploads/2020...
7011,7012,United States of America,USA,Wyoming,Wyoming,2020-04-03,"Case identification, contact tracing and relat...",Quarantine,Incoming travelers,All,NaN,statewide directive for individuals arriving i...,https://drive.google.com/file/d/1xZF2MlVstWCSp...


In [ ]:
df_states

,Date,State,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
14089,2020-11-13,Virginia,51,199262,3785
14090,2020-11-13,Washington,53,132183,2634
14091,2020-11-13,West Virginia,54,31639,565
14092,2020-11-13,Wisconsin,55,318023,2694


# Omer's 2 week range builder

In [ ]:
import numpy as np
import datetime as dt

dates = df['Date'].to_numpy() # extracting dates column to numpy array
range = dt.timedelta(days = 14) #range of time frames
time_frames = [] # time frames references to check the impact from measures taken

'''this function takes in an array of dates and returns all unique dates from
the current index moving forward within the requested range (measured in days). 
For example, if we input an array of 50 dates and ask for a range of 14. we will
get an output of 50 sub arrays each array starting at each unique index and 
iterate through all the following unique arrays with in 14 days afterward.'''

# uncomment the print line at the end of the function to see what it does
def two_week_window(dates, range):
  dates = np.unique(dates) # getting unique dates
  dates.sort() # sorting dates
  
  # converting dates from strings to datetime objects
  dates_df = [dt.datetime.strptime(date, '%Y-%m-%d').date() for date in dates]

  for i in dates_df:
    window = []
    start_date = i 
    end_date = start_date + range 
    splice = [date for date in dates_df if start_date <= date < end_date]
    window.append(splice)
    time_frames.append(window)
    print(window)
  
# time frames references to check the impact from measures taken
two_week_window(dates, range)

print(time_frames[62]) # last subarray in time_frame (63 unique dates in set)

[[datetime.date(2020, 1, 29), datetime.date(2020, 2, 2)]]
[[datetime.date(2020, 2, 2)]]
[[datetime.date(2020, 2, 27), datetime.date(2020, 3, 1), datetime.date(2020, 3, 2), datetime.date(2020, 3, 3), datetime.date(2020, 3, 4), datetime.date(2020, 3, 5), datetime.date(2020, 3, 6), datetime.date(2020, 3, 7), datetime.date(2020, 3, 8), datetime.date(2020, 3, 9), datetime.date(2020, 3, 10), datetime.date(2020, 3, 11)]]
[[datetime.date(2020, 3, 1), datetime.date(2020, 3, 2), datetime.date(2020, 3, 3), datetime.date(2020, 3, 4), datetime.date(2020, 3, 5), datetime.date(2020, 3, 6), datetime.date(2020, 3, 7), datetime.date(2020, 3, 8), datetime.date(2020, 3, 9), datetime.date(2020, 3, 10), datetime.date(2020, 3, 11), datetime.date(2020, 3, 12), datetime.date(2020, 3, 13), datetime.date(2020, 3, 14)]]
[[datetime.date(2020, 3, 2), datetime.date(2020, 3, 3), datetime.date(2020, 3, 4), datetime.date(2020, 3, 5), datetime.date(2020, 3, 6), datetime.date(2020, 3, 7), datetime.date(2020, 3, 8), datet